# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [43]:
WeatherData = "CityWeather.csv"
WeatherDF = pd.read_csv(WeatherData)
WeatherDF

,Unnamed: 0,City,Clouds,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,0,punta arenas,40,CL,1579573328,67,-53.15,-70.92,57.20,12.75
1,1,vaini,20,TO,1579573329,66,-21.20,-175.20,84.20,12.75
2,2,provideniya,75,RU,1579573330,71,64.38,-173.30,3.20,4.47
3,3,katsuura,20,JP,1579573332,40,35.13,140.30,57.99,24.16
4,4,hamilton,90,US,1579573333,68,39.18,-84.53,26.01,4.70
...,...,...,...,...,...,...,...,...,...,...
542,542,lingao,100,MG,1579573938,94,-20.28,44.28,75.00,4.43
543,543,kharp,26,CL,1579573940,91,-25.40,-70.48,61.70,0.51
544,544,yuncheng,75,MU,1579573941,88,-20.01,57.66,80.01,10.78
545,545,sangar,93,IN,1579573942,92,34.17,77.58,-3.87,3.89


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:
gmaps.configure(api_key=g_key)

In [45]:
coordinates = WeatherDF[['Lat', 'Lng']]
coordinates

,Lat,Lng
0,-53.15,-70.92
1,-21.20,-175.20
2,64.38,-173.30
3,35.13,140.30
4,39.18,-84.53
...,...,...
542,-20.28,44.28
543,-25.40,-70.48
544,-20.01,57.66
545,34.17,77.58


In [46]:
f = gmaps.figure()
humidity = WeatherDF['Humidity']
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=True)
f.add_layer(heat_layer)
f

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
zWeatherDF = WeatherDF.loc[WeatherDF['Clouds'] == 0]
yWeatherDF = zWeatherDF.loc[zWeatherDF['Max Temperature'] < 83]
xWeatherDF = yWeatherDF.loc[yWeatherDF['Max Temperature'] > 70]
wWeatherDF = xWeatherDF.loc[xWeatherDF['Humidity'] < 90]
wWeatherDF.dropna()

,Unnamed: 0,City,Clouds,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
39,39,lorengau,0,IN,1579573371,63,17.48,73.19,76.66,6.26
42,42,atuona,0,IN,1579573374,58,17.18,77.28,73.04,12.68
57,57,caravelas,0,PH,1579573391,75,16.29,119.81,78.37,13.13
79,79,upernavik,0,CL,1579573415,50,-33.73,-70.75,75.20,6.93
195,195,konigslutter,0,AU,1579573315,65,-24.87,113.63,78.80,23.04
222,222,elizabeth city,0,CU,1579573574,80,20.71,-78.00,77.09,16.33
266,266,ust-kut,0,MX,1579573625,81,22.55,-105.75,71.37,1.66
303,303,berlevag,0,AR,1579573668,61,-34.62,-68.33,74.62,14.54
369,369,nalut,0,AU,1579573743,36,-34.73,135.87,75.22,5.88
378,378,bubaque,0,AU,1579573753,47,-32.50,137.77,79.00,13.87


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
import json

In [55]:
hotel_df = wWeatherDF.dropna()
hotel_df['Hotel Name'] = ""

In [56]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": hotel_df[["Lat","Lng"]],
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key,
}
response = requests.get(base_url,params=params).json()
print(json.dumps(response, indent=4, sort_keys=True))
#for index, row in hotel_df.iterrows():
   # try:
 #       response = requests.get(base_url,params=params).json()
  #      results = response['results']
   #     print(response["results"][0]["name"])
        
   # except(KeyError, IndexError):
    #    print("N/A response. skipping")

{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}


{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [57]:
# Add marker layer ontop of heat map


# Display Map

AttributeError: 'DataFrame' object has no attribute 'dtype'